In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision as tv
from torchvision import datasets, models, transforms
from sklearn.metrics import roc_auc_score

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import time
import os
import copy
import requests
import io
import csv
import random
from timm.data import create_dataset, create_loader
from timm.scheduler import StepLRScheduler


plt.ion()   # interactive mode

import timm 
from tqdm import tqdm

In [2]:
# Check if CUDA (GPU support) is available
is_cuda_available = torch.cuda.is_available()
print("Is CUDA (GPU) available:", is_cuda_available)

# If CUDA is available, print the GPU name(s)
if is_cuda_available:
    gpu_count = torch.cuda.device_count()
    print(f"Number of GPU(s) available: {gpu_count}")
    for i in range(gpu_count):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available. Using CPU.")

Is CUDA (GPU) available: True
Number of GPU(s) available: 1
GPU 0: Quadro RTX 6000


# Prediction - Binary

In [38]:
# config 
input_size = 3, 224, 224
img_size = 224
num_classes = 2
batch_size = 128
interpolation = 'bicubic'

test_dir = '../Dataset/images/test_binary'

class_map = {
        'Normal': 0,
        'Cases': 1
        }

# create the train and etest datasets
test_dataset = create_dataset(name='', root=test_dir, split='validation', is_training=False, batch_size=batch_size, class_map = class_map)

test_len = len(test_dataset)
print('testidation set size: ' + str(test_len))

# resize images to fit the input of pretrained model
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.CenterCrop((224*3, 224*3)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
test_dataset.transform = transform

# create data loaders 
loader_test = create_loader(
        test_dataset,
        input_size=input_size,
        batch_size=batch_size,
        is_training=False,
        interpolation=interpolation,
        num_workers=8,
        pin_memory=True)

# check if labels are loaded as defined
test_dataset.reader.class_to_idx

testidation set size: 25596


{'Normal': 0, 'Cases': 1}

In [8]:
# define model
model_name = 'vit_base_patch16_224.orig_in21k'

model = timm.create_model(model_name, pretrained=False, num_classes=num_classes) #, img_size=img_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # if print 'cuda' then GPU is used
model.to(device)

cuda


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

## Load model

In [11]:
ckpt_save_path = 'model_result/model_checkpoint/ViT-Binary-2/MODEL_CKPT_29_vit_base_patch16_224.orig_in21k_20240407_152931.pt'
checkpoint = torch.load(ckpt_save_path)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [12]:
model.eval()
torch.cuda.empty_cache()

## Predict

In [25]:
test_labels = np.empty((0,))
test_predictions = np.empty((0,))

with torch.no_grad():
    for images, labels in tqdm(loader_test):
        images = images.to(device)
        labels = labels.cpu().numpy()
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        predictions = predictions.cpu().numpy()

        test_labels = np.concatenate((test_labels, labels))
        test_predictions = np.concatenate((test_predictions, predictions))

100%|██████████| 200/200 [03:04<00:00,  1.09it/s]


In [33]:
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, confusion_matrix

In [34]:
# Accuracy
accuracy = accuracy_score(test_labels, test_predictions)
print("Accuracy:", accuracy)

# AUC
auc = roc_auc_score(test_labels, test_predictions)
print("AUC:", auc)

# Recall
recall = recall_score(test_labels, test_predictions)
print("Recall:", recall)

# Precision
precision = precision_score(test_labels, test_predictions)
print("Precision:", precision)

# Confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, test_predictions).ravel()
print("True Negative:", tn)
print("False Positive:", fp)
print("False Negative:", fn)
print("True Positive:", tp)

# Sensitivity (Recall)
sensitivity = tp / (tp + fn)
print("Sensitivity:", sensitivity)

# Specificity
specificity = tn / (tn + fp)
print("Specificity:", specificity)

Accuracy: 0.696319737458978
AUC: 0.674146721410446
Recall: 0.7707658087067047
Precision: 0.7443230637044311
True Negative: 5695
False Positive: 4166
False Negative: 3607
True Positive: 12128
Sensitivity: 0.7707658087067047
Specificity: 0.5775276341141872


# Prediction - Multiclass_Caseonly_grouped

In [3]:
# config 
input_size = 3, 224, 224
img_size = 224
num_classes = 8
batch_size = 128

interpolation = 'bicubic'
DEFAULT_CROP_PCT = 1

test_dir = '../Dataset/images/test_caseonly_grouped'

class_map = {
    'Fluid_overload': 0,
    'Infection': 1,
    'Mass_Like_Lesions': 2,
    'Parenchymal_Disease': 3,
    'Atelectasis': 4,
    'Cardiomegaly': 5,
    'Pneumothorax': 6,
    'Pleural_Thickening': 7
    }

# create the train and etest datasets
test_dataset = create_dataset(name='', root=test_dir, split='validation', is_training=False, batch_size=batch_size, class_map = class_map)

test_len = len(test_dataset)
print('testidation set size: ' + str(test_len))

# resize images to fit the input of pretrained model
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.CenterCrop((224*3, 224*3)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
test_dataset.transform = transform

# create data loaders 
loader_test = create_loader(
        test_dataset,
        input_size=input_size,
        batch_size=batch_size,
        is_training=False,
        interpolation=interpolation,
        num_workers=8,
        pin_memory=True)

# check if labels are loaded as defined
test_dataset.reader.class_to_idx

testidation set size: 25726


{'Fluid_overload': 0,
 'Infection': 1,
 'Mass_Like_Lesions': 2,
 'Parenchymal_Disease': 3,
 'Atelectasis': 4,
 'Cardiomegaly': 5,
 'Pneumothorax': 6,
 'Pleural_Thickening': 7}

In [4]:
# define model
model_name = 'vit_base_patch16_224.orig_in21k'

model = timm.create_model(model_name, pretrained=False, num_classes=num_classes) #, img_size=img_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # if print 'cuda' then GPU is used
model.to(device)

cuda


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [5]:
ckpt_save_path = 'model_result/model_checkpoint/ViT-Multiclass-Caseonly-grouped/MODEL_CKPT_29_vit_base_patch16_224.orig_in21k_20240424_020015.pt'
checkpoint = torch.load(ckpt_save_path)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [6]:
model.eval()
torch.cuda.empty_cache()

## Predict

In [7]:
import torch.nn.functional as F

In [8]:
test_labels = np.empty((0,))
test_predictions = np.empty((0,))

with torch.no_grad():
    for images, labels in tqdm(loader_test):
        images = images.to(device)
        labels = labels.cpu().numpy()
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        predictions = predictions.cpu().numpy()

        test_labels = np.concatenate((test_labels, labels))
        test_predictions = np.concatenate((test_predictions, predictions))

100%|██████████| 201/201 [03:52<00:00,  1.16s/it]


In [9]:
# Get unique values and their counts
unique_values, counts = np.unique(test_labels, return_counts=True)

# Print unique values and their counts
for value, count in zip(unique_values, counts):
    print(f"{value}: {count}")

0.0: 5337
1.0: 7671
2.0: 3052
3.0: 1510
4.0: 3279
5.0: 1069
6.0: 2665
7.0: 1143


In [49]:
# Get unique values and their counts
unique_values, counts = np.unique(test_predictions, return_counts=True)

# Print unique values and their counts
for value, count in zip(unique_values, counts):
    print(f"{value}: {count}")

0.0: 5692
1.0: 11192
2.0: 2259
3.0: 1412
4.0: 2679
5.0: 1035
6.0: 1244
7.0: 213


In [73]:
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

In [55]:
# Accuracy
accuracy = accuracy_score(test_labels, test_predictions)
print("Accuracy:", accuracy)

# Confusion matrix4
conf_matrix = confusion_matrix(test_labels, test_predictions)
print("Confusion Matrix:")
print(conf_matrix)

# Classification report
class_report = classification_report(test_labels, test_predictions)
print("Classification Report:")
print(class_report)


Accuracy: 0.37506802456658633
Confusion Matrix:
[[2013 2148  250   97  469  184  139   37]
 [1447 4556  394  192  636  257  152   37]
 [ 428 1255  872  114  201   41  111   30]
 [ 164  464  125  424  132   28  151   22]
 [ 807 1098  163  114  859   92  135   11]
 [ 163  410   18   11   62  400    3    2]
 [ 367  842  297  409  228   11  481   30]
 [ 303  419  140   51   92   22   72   44]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.35      0.38      0.37      5337
         1.0       0.41      0.59      0.48      7671
         2.0       0.39      0.29      0.33      3052
         3.0       0.30      0.28      0.29      1510
         4.0       0.32      0.26      0.29      3279
         5.0       0.39      0.37      0.38      1069
         6.0       0.39      0.18      0.25      2665
         7.0       0.21      0.04      0.06      1143

    accuracy                           0.38     25726
   macro avg       0.34      0.30      0.3

In [77]:
# Convert true labels and predicted labels to binary format
test_labels_binary = label_binarize(test_labels, classes=[0, 1, 2, 3, 4, 5, 6, 7])
test_predictions_binary = label_binarize(test_predictions, classes=[0, 1, 2, 3, 4, 5, 6, 7])

# Compute OvR AUC
ovr_auc = roc_auc_score(test_labels_binary, test_predictions_binary, average='macro')
print("OvR AUC:", ovr_auc)

OvR AUC: 0.5968573393691229


In [78]:
# Compute OvR AUC for each class
class_auc_scores = []
for i in range(len(test_labels_binary[0])):
    class_test_labels = test_labels_binary[:, i]
    class_test_predictions = test_predictions_binary[:, i]
    class_auc = roc_auc_score(class_test_labels, class_test_predictions)
    class_auc_scores.append(class_auc)

# Print AUC for each class
for i, auc_score in enumerate(class_auc_scores):
    print(f"Class {i}: AUC = {auc_score:.4f}")

Class 0: AUC = 0.5984
Class 1: AUC = 0.6132
Class 2: AUC = 0.6123
Class 3: AUC = 0.5904
Class 4: AUC = 0.6742
Class 5: AUC = 0.5737
Class 6: AUC = 0.5158


# Obtain embedding

In [16]:
with torch.no_grad():
    images, labels = next(iter(loader_test))
    out = model.patch_embed(images)
    out = model.pos_drop(out)
    out = model.patch_drop(out)
    out = model.norm_pre(out)
    out = model.blocks(out)
    out = model.norm(out)
    out = model.fc_norm(out)
    out = model.head_drop(out)

In [19]:
# flatten
out = out.view(out.size(0), -1)

In [20]:
out.shape

tensor([[ 0.3829, -0.1081,  0.0073,  ..., -0.4189,  0.2057,  0.0613],
        [ 0.4378, -0.2493,  0.0121,  ..., -0.3218,  0.2583,  0.1810],
        [ 0.2535, -0.0295, -0.0203,  ..., -0.1449,  0.0451,  0.3238],
        ...,
        [ 0.4210, -0.1702,  0.1818,  ..., -0.1054,  0.1330,  0.0166],
        [ 0.2879, -0.0817,  0.1365,  ..., -0.2921, -0.0364, -0.2090],
        [ 0.2229,  0.0106,  0.1802,  ..., -0.3411, -0.0572, -0.0428]],
       device='cuda:0')